In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [9]:
import os
import numpy as np
import pandas as pd

In [25]:
df = pd.read_csv("data/structure/secondary_structure/tape_ss3.csv")
df["ss3"] = df["ss3"].apply(lambda x: np.array(x[1:-1].split(", ")))
df.head()

,sequence,ss3,split
0,AETVESCLAKSHTENSFTNVXKDDKTLDRYANYEGCLWNATGVVVC...,"[2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, ...",train
1,ASQEISKSIYTCNDNQVXEVIYVNTEAGNAYAIISQVNEXIPXRLX...,"[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, ...",train
2,XGSSHHHHHHSSGRENLYFQGXNISEINGFEVTGFVVRTTNADEXN...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",train
3,SNADYNRLSVPGNVIGKGGNAVVYEDAEDATKVLKMFTTSQSNEEV...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...",train
4,DTVGRPLPHLAAAMQASGEAVYCDDIPRYENELFLRLVTSTRAHAK...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, ...",train


In [26]:
df.sequence[0], df.ss3[0], 

('AETVESCLAKSHTENSFTNVXKDDKTLDRYANYEGCLWNATGVVVCTGDETQCYGTWVPIGLAIPENEGGGSEGGGSEGGGSEGGGTKPPEYGDTPIPGYTYINPLDGTYPPGTEQNPANPNPSLEESQPLNTFMFQNNRFRNRQGALTVYTGTVTQGTDPVKTYYQYTPVSSKAMYDAYWNGKFRDCAFHSGFNEDIFVCEYQGQSSDLPQPPVNA',
 array(['2', '2', '2', '0', '0', '0', '0', '0', '2', '2', '2', '2', '1',
        '1', '1', '1', '1', '1', '2', '2', '1', '1', '2', '2', '2', '2',
        '2', '2', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '1',
        '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '1',
        '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2',
        '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
        '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
        '2', '2', '2', '2', '2', '1', '1', '1', '1', '1', '2', '2', '2',
        '2', '2', '2', '2', '2', '2', '2', '1', '2', '2', '2', '2', '1',
        '2', '2', '2', '2', '2', '2', '1', '1', '1', '2', '2', '2', '2',
        '2', '2', '2', '1', '1', '1', '2', '2', 

In [27]:
len(df.sequence[0]), len(df.ss3[0]), len(df.sequence[1]), len(df.ss3[1])

(217, 217, 98, 98)

In [7]:
df.split.unique()

array(['train', 'valid', 'cb513', 'ts115', 'casp12'], dtype=object)

In [28]:
from scr.preprocess.data_process import std_ssdf

In [29]:
std_ssdf()

In [41]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import numpy as np

X, y1 = make_classification(n_samples=10, n_features=100,
                            n_informative=30, n_classes=3,
                            random_state=1)
y2 = shuffle(y1, random_state=1)
y3 = shuffle(y1, random_state=2)
Y = np.vstack((y1, y2, y3)).T
n_samples, n_features = X.shape # 10,100
n_outputs = Y.shape[1] # 3
n_classes = 3
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=2)
multi_target_forest.fit(X, Y).predict(X)

array([[2, 2, 0],
       [1, 2, 1],
       [2, 1, 0],
       [0, 0, 2],
       [0, 2, 1],
       [0, 0, 2],
       [1, 1, 0],
       [1, 1, 1],
       [0, 0, 2],
       [2, 0, 0]])

In [43]:
from scr.encoding.encoding_classes import ESMEncoder

In [56]:
mean_batch_encoder = ESMEncoder(
    encoder_name="esm1_t6_43M_UR50S",
).encode(mut_seqs=list(df.sequence), batch_size=4, flatten_emb=False)
one_mean_emb = next(mean_batch_encoder)
one_mean_emb[0].shape

Generating esm1_t6_43M_UR50S upto 6 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main
  0%|          | 0/2875 [00:00<?, ?it/s]

No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening


(4, 217, 768)

In [57]:
X = one_mean_emb[0]
X.shape

(4, 217, 768)

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [51]:
df.ss3[:4]

0    [2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, ...
1    [2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, ...
2    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
3    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...
Name: ss3, dtype: object

In [54]:
Y = df.ss3[:4].to_numpy()
Y.shape

(4,)